# 載入函式庫與資料集

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
np.random.seed(10)  # 指定亂數種子
# 載入波士頓房屋資料集
df = pd.read_csv("./boston_housing.csv")

In [3]:
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
df.shape

(506, 14)

# 資料前處理

In [5]:
dataset = df.values
np.random.shuffle(dataset)  # 使用亂數打亂資料

# 分割成特徵資料和標籤資料
X = dataset[:, 0:13]
Y = dataset[:, 13]

# 特徵標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)

# 分割訓練和測試資料集
X_train, Y_train = X[:404], Y[:404]  # 訓練資料前404筆
X_test, Y_test = X[404:], Y[404:]   # 測試資料後102筆

# 打造模型

In [6]:
# 定義深度模型
def build_deep_model():
  model = Sequential()
  model.add(Dense(32, input_shape=(X_train.shape[1],), activation="relu"))
  model.add(Dense(16, activation="relu"))     #多加一層隱藏層 16個神經元
  model.add(Dense(1))
  # 編譯模型
  model.compile(loss="mse", optimizer="adam", metrics=["mae"])
  return model

# 使用交叉驗證

In [7]:
k = 4
nb_val_samples = len(X_train) // k
nb_epochs = 80
mse_scores = []
mae_scores = []
for i in range(k):
  print("Processing Fold #" + str(i))
  # 取出驗證資料集
  X_val = X_train[i*nb_val_samples: (i+1)*nb_val_samples]
  Y_val = Y_train[i*nb_val_samples: (i+1)*nb_val_samples]
  # 結合出訓練資料集
  X_train_p = np.concatenate(
          [X_train[:i*nb_val_samples],
          X_train[(i+1)*nb_val_samples:]], axis=0)
  Y_train_p = np.concatenate(
          [Y_train[:i*nb_val_samples],
          Y_train[(i+1)*nb_val_samples:]], axis=0)
  model = build_deep_model()
  # 訓練模型
  model.fit(X_train_p, Y_train_p, epochs=nb_epochs, 
            batch_size=16, verbose=0)
  # 評估模型
  mse, mae = model.evaluate(X_val, Y_val, verbose=0)
  mse_scores.append(mse)
  mae_scores.append(mae)

Processing Fold #0
Processing Fold #1
Processing Fold #2
Processing Fold #3


In [8]:
#交叉驗證後模型的平均指標
print("MSE_val: ", np.mean(mse_scores))
print("MAE_val: ", np.mean(mae_scores))

# 使用測試資料評估模型
mse, mae = model.evaluate(X_test, Y_test, verbose=0)    
print("MSE_test: ", mse)
print("MAE_test: ", mae)

MSE_val:  12.077487230300903
MAE_val:  2.471786618232727
MSE_test:  24.400314331054688
MAE_test:  2.8596980571746826
